In [1]:
#from odc.stac import configure_s3_access
#configure_s3_access(cloud_defaults=True, aws_unsigned=True)
from pystac_client import Client
from odc.stac import load
import fsspec
import pandas
from planetary_computer import sign_url
import math

## having trouble installing ... (psutils, psycog2-binary ...)
#from odc.algo import mask_cleanup


url = "https://github.com/mdsumner/nuyina.underway/raw/main/data-raw/nuyina_underway.parquet"
with fsspec.open(url) as f: 
    df = pandas.read_parquet(f)
   



In [ ]:
d0 = df.tail(1)

##pt = [77.93677, -68.56996]
pt = [d0.longitude.to_list(), d0.latitude.to_list()]

## distance around point in metres
ds = 500
b = ds / (1852 * 60)
a = b * math.cos(pt[1] * math.pi/180)
print(a)
bbox = [pt[0] - a, pt[1] - b, pt[0] + a, pt[1] + b]
 

client = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1/")

result = client.search(collections = ["sentinel-2-l2a", "landsat-c2-l2"], bbox = bbox, datetime = "2023-11")

items = list(result.items())
print(len(items))

items[0].to_dict()

data = load(items, bbox = bbox, chunks = {}, groupby = "solar_day", measurements = ["red", "green", "blue"], patch_url=sign_url, crs = "EPSG:32743", fail_on_error=False)


print(data.x.values[1] - data.x.values[0])

In [59]:


data[["red", "green", "blue"]].to_array().plot.imshow(robust=True, col = "time", col_wrap = 2, size = 4)


30.0


In [54]:
#CLOUD = 3
#CLOUD_SHADOW = 4

#bitmask = 0
#for field in [CLOUD_SHADOW, CLOUD]:
#    bitmask |= 1 << field


#cloud_mask = data.qa_pixel.astype("uint16") & bitmask != 0

In [55]:
#cloud_mask.plot.imshow(col = "time", col_wrap = 2, size = 6)